# Python Decorators (파이썬 데코레이터)

## 데코레이터란?

**데코레이터(`Decorator`)**는 기존 함수의 코드를 **수정하지 않고** 추가적인 동작(로깅, 권한 확인, 시간 측정 등)을 **추가**해주는 기능입니다.

데코레이터는 **함수를 입력으로 받아 새로운 함수를 반환**하는 함수입니다.

### 기본 구조

1.  데코레이터 함수를 정의합니다. (외부 함수는 `func`를 인수로 받음)
2.  데코레이터 내부에 **래퍼 함수(`Wrapper Function`)**를 정의합니다. (이 함수가 실제 추가 동작을 수행)
3.  데코레이터 함수는 래퍼 함수를 반환합니다.
4.  사용할 함수 위에 **`@데코레이터_이름`** 구문을 붙여 적용합니다.

In [1]:
# 예제 1: 기본 데코레이터 (반환 값을 대문자로 변경)

def changecase(func):
  # 래퍼 함수 (실제 데코레이팅 동작 수행)
  def myinner():
    return func().upper()
  return myinner

@changecase # myfunction = changecase(myfunction)과 동일
def myfunction():
  return "Hello Sally"

print(myfunction())

HELLO SALLY


## Multiple Decorator Calls (다중 데코레이터 호출)

하나의 데코레이터를 여러 함수에 반복하여 적용할 수 있습니다. `@데코레이터_이름`을 원하는 함수 위에 붙여주기만 하면 됩니다.

In [2]:
# 예제 2: 두 함수에 동일한 데코레이터 적용
def changecase(func):
  def myinner():
    return func().upper()
  return myinner

@changecase
def myfunction():
  return "Hello Sally"

@changecase
def otherfunction():
  return "I am speed!"

print(myfunction())
print(otherfunction())

HELLO SALLY
I AM SPEED!


## Arguments in the Decorated Function (인수를 가진 함수 데코레이팅)

데코레이트될 함수가 인수를 필요로 한다면, 데코레이터 내부의 **래퍼 함수(`myinner`)**도 해당 인수를 받아 원래 함수에 전달해야 합니다.

In [3]:
# 예제 3: 인수를 가진 함수 데코레이팅
def changecase(func):
  # 래퍼 함수가 x 인수를 받아서
  def myinner(x):
    # 원래 함수 func(x)에 전달
    return func(x).upper()
  return myinner

@changecase
def myfunction(nam):
  return "Hello " + nam

print(myfunction("John"))

HELLO JOHN


## `*args` and `**kwargs`로 유연성 확보

데코레이터는 데코레이트될 함수가 **어떤 인수를 얼마나 받을지** 모를 때가 많습니다. 이 문제를 해결하기 위해 래퍼 함수에 **`*args`와 `**kwargs`**를 추가하여 모든 종류의 인수를 받아 원래 함수에 전달합니다.

이를 통해 범용적인 데코레이터를 만들 수 있습니다.

In [4]:
# 예제 4: *args와 **kwargs를 사용하여 모든 인수에 대응
def changecase(func):
  # 래퍼 함수가 가변 인자를 받아
  def myinner(*args, **kwargs):
    # 이를 그대로 원래 함수에 전달
    return func(*args, **kwargs).upper()
  return myinner

@changecase
def myfunction(nam): # 인수가 1개든 10개든 작동
  return "Hello " + nam

print(myfunction("John"))

HELLO JOHN


## Decorator With Arguments (인수를 받는 데코레이터)

데코레이터 자체가 인수를 받아야 할 경우, **함수를 한 번 더 중첩**하여 **데코레이터 팩토리(`Decorator Factory`)** 형태로 만듭니다.

* **가장 바깥 함수:** 데코레이터의 인수를 받음 (`n`)
* **중간 함수:** 함수(`func`)를 인수로 받는 실제 데코레이터
* **가장 안쪽 함수:** 래퍼 함수 (`myinner`)

In [5]:
# 예제 5: 데코레이터 인수에 따라 대소문자 변경 방식 결정
def changecase(n): # 1. 데코레이터 인수 (n)를 받음
  def actual_decorator(func): # 2. 데코레이트될 함수 (func)를 받음
    def myinner(*args, **kwargs): # 3. 래퍼 함수
      if n == 1:
        a = func(*args, **kwargs).lower() # n=1이면 소문자로
      else:
        a = func(*args, **kwargs).upper() # n=1이 아니면 대문자로
      return a
    return myinner
  return actual_decorator

@changecase(1) # 데코레이터 호출 시 인수를 전달 (n=1)
def myfunction():
  return "Hello Linus"

print(myfunction())

hello linus


## Multiple Decorators (다중 데코레이터)

하나의 함수에 여러 개의 데코레이터를 적용할 수 있습니다. 데코레이터 호출을 위아래로 쌓아 올립니다.

**✅ 호출 순서:**

* 데코레이터는 **함수와 가장 가까운 데코레이터부터** (아래에서 위로) **역순으로** 호출됩니다.

In [6]:
# 예제 6: 대문자 변경 데코레이터와 인사말 추가 데코레이터 결합
def changecase(func):
  # 1. 가장 안쪽에서 먼저 실행됨 (@addgreeting의 결과를 받음)
  def myinner():
    return func().upper()
  return myinner

def addgreeting(func):
  # 2. 두 번째로 실행됨 (myfunction의 결과를 받음)
  def myinner():
    return "Hello " + func() + " Have a good day!"
  return myinner

@changecase # 최종 결과에 적용 (가장 바깥)
@addgreeting # 먼저 실행됨 (가장 안쪽)
def myfunction():
  return "Tobias"

# 결과: "Hello Tobias Have a good day!"가 먼저 적용되고, 최종적으로 .upper()가 적용됨
print(myfunction())

HELLO TOBIAS HAVE A GOOD DAY!


## Preserving Function Metadata (함수 메타데이터 보존)

파이썬 함수는 `__name__`, `__doc__`와 같은 메타데이터 속성을 가지고 있습니다.

함수를 데코레이터로 감싸면, 함수 이름(`__name__`)이 원래 함수 이름 대신 **래퍼 함수(`myinner`)의 이름으로 대체**되는 문제가 발생합니다.

이 문제를 해결하기 위해 **`functools.wraps`** 데코레이터를 사용해야 합니다.

In [7]:
# 예제 7: 데코레이터 사용 시 메타데이터 손실 확인
def changecase(func):
  def myinner():
    return func().upper()
  return myinner

@changecase
def myfunction():
  """원래 함수의 설명입니다."""
  return "Have a great day!"

# 데코레이팅 후 __name__을 확인하면 'myinner'가 나옴 (손실 발생)
print(f"손실된 함수 이름: {myfunction.__name__}")

손실된 함수 이름: myinner


In [8]:
# 예제 8: functools.wraps를 사용하여 메타데이터 보존
import functools

def changecase(func):
  # @functools.wraps(func)를 래퍼 함수 위에 적용
  @functools.wraps(func) 
  def myinner():
    return func().upper()
  return myinner

@changecase
def myfunction():
  """이것이 원래 함수의 docstring입니다."""
  return "Have a great day!"

# 데코레이팅 후 __name__을 확인하면 'myfunction'이 정상적으로 나옴
print(f"보존된 함수 이름: {myfunction.__name__}")
print(f"보존된 docstring: {myfunction.__doc__}")

보존된 함수 이름: myfunction
보존된 docstring: 이것이 원래 함수의 docstring입니다.
